# Try out CNN on EEG data with labels

## Pre-processing
+ Import data.
+ Apply filters (bandpass).
+ Detect potential bad channels and replace them by interpolation.
+ Detect potential bad epochs and remove them.

## Train CNN network
+ Define network architecture
+ Split data
+ Train model

### Use mcfly for some first model testing: https://github.com/NLeSC/mcfly

## Import packages & links

In [3]:
# Import packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import mne
#%matplotlib inline

from mayavi import mlab

In [4]:
ROOT = "C:\\OneDrive - Netherlands eScience Center\\Project_ePodium\\"
PATH_CODE = ROOT + "EEG_explorer\\"
PATH_DATA = ROOT + "Data\\"
PATH_OUTPUT = ROOT + "Data\\processed\\"
PATH_METADATA = ROOT + "Data\\metadata\\"
file_labels = "metadata.xlsx"

import sys
sys.path.insert(0, PATH_CODE)

## Load pre-processed dataset
+ See notebook for preprocessing: Exploring_EEG_data_04_prepare_data_for_ML.ipynb

In [5]:
filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected.npy"
signal_collection = np.load(filename)

filename = PATH_OUTPUT + "EEG_data_30channels_1s_corrected_labels.npy"
label_collection = np.load(filename)

## Split training data
The entire dataset is split into:
+ training data (here: about 70%) which is used to train a model.
+ validation data, used to monitor the model progress and avoid overfitting.
+ testing data, meant for final check on model performance.

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(signal_collection, label_collection, test_size=0.15, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

In [7]:
print('Train set size:', X_train.shape[0])
print('Validation set size:', X_val.shape[0])
print('Test set size:', X_test.shape[0])

Train set size: 28237
Validation set size: 4983
Test set size: 5863


## Switch to 1-hot encoding for labels
+ We have six categories or classes. Those are best represented by a so called 1-hot encoding. This means nothing else than simply a binary 0-or-1 for every class.. 

In [8]:
from sklearn.preprocessing import LabelBinarizer
label_transform = LabelBinarizer()

y_train_binary = label_transform.fit_transform(y_train.astype(int))
y_val_binary = label_transform.fit_transform(y_val.astype(int))
y_test_binary = label_transform.fit_transform(y_test.astype(int))

In [9]:
y_val_binary[:10,:]

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [10]:
# Show found labels:
label_transform.classes_

array([  3,   6,  13,  26,  66, 132])

Check distribution accross the 6 label categories:

In [11]:
labels = list(label_transform.classes_)
frequencies = y_train_binary.mean(axis=0)
frequencies_df = pd.DataFrame(frequencies, index=labels, columns=['frequency'])
frequencies_df

,frequency
3,0.313206
6,0.488437
13,0.038885
26,0.061515
66,0.038141
132,0.059815


### Note:
We have more data on group 2 than on group 1. And far more data for stimuli 3 than for stimuli 13 and 66 (not surprising). 

--> post on balancing datasets: https://towardsdatascience.com/handling-imbalanced-datasets-in-deep-learning-f48407a0e758

### Needs some thinking on how to balance the data set !
e.g. by frequency dependend selection rule, or by defining a suitied special loss function....

In [ ]:
labels
1/frequencies

class_weight = {"buy": 0.75,
                "don't buy": 0.25}

model.fit(X_train, Y_train, epochs=10, batch_size=32, class_weight=class_weight)

## Define model architecture

In [ ]:
n_timesteps = 501
n_features = 30

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])